In [1]:
import torch
from modules import Simple_Perceptron
from data import grip_data

In [2]:
model = Simple_Perceptron.Simple_Perceptron(41, 100, 1)
train_loader, X_train, Y_train, X_test, Y_test, D = grip_data.load_data(device='cpu')
X_train, Y_train

(tensor([[-1.1173,  0.4734, -0.1599,  ..., -1.0884,  1.2663,  1.0000],
         [ 0.9199, -1.3786, -0.1887,  ...,  0.9699, -0.9360,  1.0000],
         [ 0.0318,  0.1921, -0.0224,  ...,  0.4643,  0.4142,  1.0000],
         ...,
         [ 0.4563,  0.0300, -1.4275,  ...,  0.6309, -1.3905,  1.0000],
         [-1.7183, -1.6230, -0.1612,  ...,  0.0225, -0.5320,  1.0000],
         [ 1.4654, -0.4437,  0.6230,  ..., -1.0231, -0.7424,  1.0000]]),
 tensor([ 0.3786,  0.2264, -1.5667,  ..., -0.3799,  1.3942,  0.5954]))

In [4]:
lr = 1
for X, Y in train_loader:
    U = (model(X) - Y.reshape(-1, 1))
    # 增广模型中的参数
    theta_0 = torch.cat([model.W.flatten(), model.a.flatten()]).reshape(-1, 1)
    # 计算雅可比矩阵
    J = torch.zeros(U.shape[0], theta_0.numel())
    for i in range(U.shape[0]):
        U[i].backward(retain_graph=True)
        J[i] = torch.cat([model.W.grad.flatten(), model.a.grad.flatten()])
        model.W.grad.zero_()
        model.a.grad.zero_()
    with torch.no_grad():
        J_T = J.T
        # 计算量A，A=I + 2(J^T)J
        A = torch.eye(theta_0.numel()) + 2 * torch.mm(J_T, J)
        A_inv = torch.inverse(A)
        theta_1 = theta_0 - 2 * lr * torch.mm(torch.mm(A_inv, J_T), U)
        update_U = torch.eye(U.shape[0]) - 2 * lr * torch.mm(J, torch.mm(A_inv, J_T))
        U = torch.mm(update_U, U)
        # 更新参数
        model.W.data = theta_1[:model.W.numel()].reshape(model.W.shape)
        model.a.data = theta_1[model.W.numel():].reshape(model.a.shape)
        model.W.grad.zero_()
        model.a.grad.zero_()  

torch.Size([64, 1])
torch.Size([4200, 1])
torch.Size([64, 64]) torch.Size([64, 64])
torch.Size([64, 1])
torch.Size([4200, 1])
torch.Size([64, 64]) torch.Size([64, 64])
torch.Size([64, 1])
torch.Size([4200, 1])
torch.Size([64, 64]) torch.Size([64, 64])
torch.Size([64, 1])
torch.Size([4200, 1])
torch.Size([64, 64]) torch.Size([64, 64])
torch.Size([64, 1])
torch.Size([4200, 1])
torch.Size([64, 64]) torch.Size([64, 64])
torch.Size([64, 1])
torch.Size([4200, 1])
torch.Size([64, 64]) torch.Size([64, 64])
torch.Size([64, 1])
torch.Size([4200, 1])
torch.Size([64, 64]) torch.Size([64, 64])
torch.Size([64, 1])
torch.Size([4200, 1])
torch.Size([64, 64]) torch.Size([64, 64])
torch.Size([64, 1])
torch.Size([4200, 1])
torch.Size([64, 64]) torch.Size([64, 64])
torch.Size([64, 1])
torch.Size([4200, 1])
torch.Size([64, 64]) torch.Size([64, 64])
torch.Size([64, 1])
torch.Size([4200, 1])
torch.Size([64, 64]) torch.Size([64, 64])
torch.Size([64, 1])
torch.Size([4200, 1])
torch.Size([64, 64]) torch.Size([6

KeyboardInterrupt: 